<h1>Introduction notebook Brainpowered course</h1>

This notebook is an introductionary notebook for the course Brainpowered. In this notebook some basic data analysis on EEG data will be done. Also some data preprocessing will be done in the form of a bandwidth and notch filter will be done. At last some classification methods such as Knn (K nearest neighbors), SVM (Support Vector Machines) and LDA (Linear Discriminant Analysis). The EEG data used in this notebook is data of people who keep their eyes closed or open for a period of 10 seconds. So the classification methods are used to classify if a person has their eyes closed or open.


In [ ]:
# add your own imports if needed
import os
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from collections import defaultdict
from mne.decoding import CSP
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

### Table of contents:
* [Dataset](#dataset)
* [Making your first plots](#first_plot)
* [EEG preprocessing](#preprocessing)
* [Spectral analysis](#spectrum)
* [Analyze the two classes](#two_classes)
* [Feature extraction](#feature)
* [Classification of the two classes](#classification)


<h2>Dataset</h2> <a class="dataset" id="dataset"></a>
During this fase, we will look at the dataset. First we load the file and look at the data. The data is stored in a .mat file. This is a file format used by Matlab. We can load this file using the scipy.io library. This library is used to load matlab files. The data is stored in a dictionary. The keys of the dictionary are the names of the variables in the matlab file. The values of the dictionary are the values of the variables in the matlab file. 

It is important to look at the dataset before you start analyzing it. This way you know what you are working with. You can look at the shape of the data, the type of the data and the values of the data.

In [ ]:
data_directory = 'data_alpha/'

# Get the list of subjects in the data, and sort the list
subject_files = os.listdir(data_directory)
subject_files.sort()

# What is the structure of one .mat file?
print(scipy.io.loadmat(data_directory + subject_files[0]))

# What is the shape of the data matrix?
print(scipy.io.loadmat(data_directory + subject_files[0])["SIGNAL"].shape)

# the channel names (in order)
channel_names = ['Fp1', 'Fp2', 'Fc5', 'Fz', 'Fc6', 'T7', 'Cz', 'T8', 'P7', 'P3', 'Pz', 'P4','P8', 'O1', 'Oz', 'O2']

# the number of channels
n_channels = len(channel_names)

<h2>Making your first plots</h2> <a class="first_plot" id="first_plot"></a>
To get an idea about what the data looks like, plotting the data is a good way to do this. But first we choose the subject, and search for the data of that subject in the dataset. 

In [ ]:
# choose the subject to analyze
nth_subject = 2
subject = subject_files[nth_subject]

# now load the data from the chosen subject
subject_matrix = scipy.io.loadmat(data_directory + subject)["SIGNAL"]

To make a simple plot of our data, we use the matplotlib library. This library is used to make plots. We can plot the data using the plot function. The plot function takes two arguments, the x values and the y values. The x values are the time values and the y values are the micro voltage values.

In [ ]:
# condition 1: eyes are closed
condition_1 = subject_matrix[:, 17]

# find the indices where condition 1 is 1
idx_condition_1 = np.where(condition_1 == 1)[0]


# condition 2: eyes are open
condition_2 = subject_matrix[:, 18]

# find the indices where condition 2 is 1
idx_condition_2 = np.where(condition_2 == 1)[0]


# start from when the first condition is measured
start_idx = idx_condition_1[0]
time_stamps = subject_matrix[start_idx:, 0]
eeg_measurements = subject_matrix[start_idx:, 1:17]

# plot the EEG measurements of each person in different plots,
# where the subplots are the waves from the electrodes of one person
fig, axs = plt.subplots(n_channels, sharex=True)

# adjust sizes of plot
fig.set_figheight(30)
fig.set_figwidth(20)
plt.subplots_adjust(hspace=1.0)
print(f"{subject} EEG measurements raw data")


# plot each channel in a separate subplot
for nth_channel in range(n_channels):
    axs[nth_channel].plot(time_stamps, eeg_measurements[:, nth_channel])
    axs[nth_channel].set_title(channel_names[nth_channel])

for ax in axs.flat:
    ax.set(xlabel='Timestamps', ylabel='Voltage (uV)')

plt.show()

<h2>EEG preprocessing</h2> <a class="preprocessing" id="preprocessing"></a>

EEG preprocessing is important. EEG is a non-invasive method, so there will be many artifacts in the data. There are two types of preprocessing steps There are many preprocessing methods, but in this notebook we will keep it simple. The preprocessing steps used in this notebook are first filtering our EEG data with a notch and a bandpass filter. The notch filter is used to filter out the 50 Hz frequency which is caused by the electronics around you. The bandpass filter will be used to filter the alpha band, which is from around 8-14 Hz. These preprocessing steps can also be used during the real time flying of the drone with your own EEG data. But another preprocessing step which can be done before choosing which channels you want to measure with EEG is removal of faulty channels. We will do this preprocessing step later on in the notebook.

When applying functions/filters on your EEG data it is important to know what sampling rate your EEG data is. Otherwise the function/filter can behave in unexpected ways.

In [ ]:
# the sampling frequency used for this specific dataset
fs = 512

First we will make the notch filter for the 50 Hz artifact in EEG data. This will remove the 50 Hz and its harmonics. 

In [ ]:
# frequency you want to remove
f_notch = 50
# the quality factor of the notch filter
Q = 30
# also in this function always give your sampling frequency to the function
b, a = signal.iirnotch(f_notch, Q, fs)

When somoeone has their eyes closed the alpha wave is likely to occur in the EEG. This peak is between 8-13 Hz. So our bandpass filter will only let these frequency bands through. The parameters you need to give to the butter filter is N which is the order. The filter order controls how sharply it separates the bandpass from the stop band. So after you applied the filter and made the spectrum plots see what happens if you change this value. The higher the order the longer the filter takes. The Wn parameter is an array with the critical frequencies, if you give your sampling frequency to this function signal.butter will normalize these critical frequencies based on the sampling frequency given as input. Otherwise the filter will behave unexpectedly. The btype parameter is the type of the filter which is bandpass. Output is sos which is used for general-purpose filtering. The fs is the sampling frequency.

In [ ]:

sos = signal.butter(N=10, Wn=[8, 15], btype="bandpass", output="sos", fs=fs)
# print the filter to see how it changes with the order
print(sos)

After these filters are applied on the channels the channels can be plotted to see how they look after filtering. 

In [ ]:
subject_matrix = scipy.io.loadmat(data_directory + subject)["SIGNAL"]

condition_1 = subject_matrix[:, 17]
idx_condition_1 = np.where(condition_1 == 1)[0]
condition_2 = subject_matrix[:, 18]
idx_condition_2 = np.where(condition_2 == 1)[0]

# the timestamps from the start of condition 1 are taken, because from there the
# first task has started
time_stamps = subject_matrix[idx_condition_1[0]:, 0]
# columns 1-16 are the columns with the EEG data
eeg_measurements = subject_matrix[idx_condition_1[0]:, 1:17]

fig, axs = plt.subplots(n_channels, sharex=True)
fig.set_figheight(30)
fig.set_figwidth(20)
plt.subplots_adjust(hspace=1.0)
plt.title(f"Different channels EEG data {subject} after applying a notch and bandpass filter")
print(f"{subject} EEG measurements filtered data")

for nth_channel in range(n_channels):
    # filter the eeg measurement for the channel separately
    filtered_eeg = signal.sosfilt(sos, eeg_measurements[:, nth_channel])

    # plot the filtered eeg signal
    axs[nth_channel].plot(time_stamps[idx_condition_1[0]:], filtered_eeg[idx_condition_1[0]:])
    axs[nth_channel].set_title(channel_names[nth_channel])

for ax in axs.flat:
    ax.set(xlabel='Timestamps', ylabel='Voltage (uV)')
plt.show()

You can see that the milivolt amplitude changes based on the filter. Also the data is more centered around zero. This doesn't really show what the filters achieved. For that we need to look at the spectrums. Which will be done in the next section.

The helper function below can be used to extract columns from the data.

In [ ]:
# a helper function to extract different columns from the data
def extract_columns(filename, path_to_file):
    '''
        Function to extract the columns from the .mat file of a subject

        Parameters
        ----------
        filename : str
            The filename of the person you want to extract the columns from.
        path_to_file : str
            The path to where the .mat file is stored.

        Returns
        -------
        idx_condition_1 : arr
            The array with indices for condition 1.
        idx_condition_2 : arr
            The array with indices for condition 2.
        time_stamps : arr
            Array with the timestamps of the measurements.
        eeg_measurements : ndarray
            A matrix where each channel is an electrode of the EEG and the
            rows are the measurements.
    '''

    subject_data = scipy.io.loadmat(path_to_file + filename)["SIGNAL"]
    # condition 1 is in column 17
    condition_1 = subject_data[:, 17]
    # index for starting row of condition 1
    idx_condition_1 = np.where(condition_1 == 1)[0]
    # condition 2 is in column 18
    condition_2 = subject_data[:, 18]
    # index for starting row of condition 1
    idx_condition_2 = np.where(condition_2 == 1)[0]

    time_stamps = subject_data[:, 0]
    eeg_measurements = subject_data[:, 1:17]

    return (idx_condition_1, idx_condition_2, time_stamps, eeg_measurements)

Before we look at the spectra we first need to separate the trials of the different classes. Above we plotted the whole recording (except for the timestamps before condition 1), but our scope is to classify if someone has their eyes closed (condition 1) or open (condition 2). So another step in the preprocessing is the data prepartion where we will store the data of the different classes in a dictionary/other datastructure of your choice. Each block of the measurements consisted of 10 seconds (which can be read on the website) of recorder EEG data and 10 blocks were measured, so 10 trials. With the sample frequency which is 512 samples per second we can calculate that the next 10 * 512 samples (indices in our array) are part of the block.

In [ ]:
# choose the trial you want to visualize because otherwise there will be too
# many plots (for training you will use all trials)
nth_trial = 0

# These are the channels used in the analysis. Change them to experiment and see
# which channels are the best for this specific task and data
channel_names = ['Fp1', 'Fp2', 'Fc5', 'Fz', 'Fc6', 'T7', 'Cz', 'T8', 'P7', 'P3', 'Pz', 'P4','P8', 'O1', 'Oz', 'O2']
# channel_names = ['Fp1', 'Fp2', 'Fc5', 'Fz', 'Fc6', 'T7', 'Cz']

In [ ]:
# extract the data and different columns of the specified subject stored in the variable "subject"
idx_condition_1, idx_condition_2, time_stamps, eeg_measurements = extract_columns(subject, data_directory)

# initialize all the dictionaries. With default dict you don't need to check if
# the key already exists, so less if statements
# the key of this dictionary will be the channel name and after that the different
# trials are appended, so the structure of the dictionaries will be:
# {channel_name1 : [trial1, trial2,..., trialn], channel_name2 : [trial1,...,trialn]}
condition_1_raw = defaultdict(lambda: [])
condition_2_raw = defaultdict(lambda: [])
condition_1_filt = defaultdict(lambda: [])
condition_2_filt = defaultdict(lambda: [])

length_measured_block = 10 # in seconds

for nth_channel in range(len(channel_names)):

    channel_data = eeg_measurements[:, nth_channel]

    for idx_1, idx_2 in zip(idx_condition_1, idx_condition_2):
        # add the + 1 because of how slicing works
        condition_1_raw[channel_names[nth_channel]].append(channel_data[idx_1:idx_1 + fs * length_measured_block + 1])
        condition_1_filt[channel_names[nth_channel]].append(signal.lfilter(b, a, signal.sosfilt(sos, channel_data)[idx_1:idx_1 + fs * length_measured_block + 1]))

        condition_2_raw[channel_names[nth_channel]].append(channel_data[idx_2:idx_2 + fs * length_measured_block + 1])
        condition_2_filt[channel_names[nth_channel]].append(signal.lfilter(b, a, signal.sosfilt(sos, channel_data)[idx_2:idx_2 + fs * length_measured_block + 1]))

In [ ]:
# check the shape of the data:
# the first dimension is the number of trials, the second dimension is the amount of datapoints
print(np.shape(condition_1_raw[channel_names[0]]))

<h2>Spectral analysis</h2> <a class="spectrum" id="spectrum"></a>

In this notebook we will plot the spectrums in two kind of ways. With the use of the FFT and Welch's method. FFT is the most standard way to make a spectral decomposition of the data, but because EEG data is non-stationary it won't show the spectrums as smooth. So that is why we will also look at the spectrum made with Welch's method. 

### FFT


The Fast Fourier Transform (FFT) is used to transform EEG data from the time domain to the frequency domain, so we can look at frequencies present in the EEG data. This means that the raw signal is being decomposed into sinusoids of different frequencies. The frequency bands are an important feature in EEG data, which means that the FFT is an important tool to use in EEG data analysis. We will use the fft functions from numpy. The function which returns the frequencies returns positive and negative frequencies, which arises from the mathematics of the Fourier transform. We are only interested in the magnitude of the frequency components, so that is why we will filter to get only the positive frequencies.

To show the result of the FFT we will plot the EEG data of the nth trial of the specified person, filtered (FFT) vs. unfiltered (raw):

In [ ]:
fig, axs = plt.subplots(len(channel_names), 2, figsize=(15, 3 *len(channel_names)))
for i, channel in enumerate(channel_names):

    len_data = len(condition_1_raw[channel][nth_trial])
    fft_vals = np.fft.fft(condition_1_raw[channel][nth_trial])
    fft_freqs = np.fft.fftfreq(len_data, 1/fs)

    # take only the positive values of the fft
    positive_freq_mask = fft_freqs > 0
    fft_freqs = fft_freqs[positive_freq_mask]
    fft_vals = fft_vals[positive_freq_mask]

    axs[i, 0].plot(fft_freqs, np.abs(fft_vals))

    len_data = len(condition_2_raw[channel][nth_trial])
    fft_vals = np.fft.fft(condition_2_raw[channel][nth_trial])
    fft_freqs = np.fft.fftfreq(len_data, 1/fs)
    # take only the positive values of the fft
    positive_freq_mask = fft_freqs > 0
    fft_freqs = fft_freqs[positive_freq_mask]
    fft_vals = fft_vals[positive_freq_mask]

    axs[i, 0].plot(fft_freqs, np.abs(fft_vals))

    axs[i, 0].set_xlim(0, 80)
    axs[i, 0].set_title(channel)


    len_data = len(condition_1_filt[channel][nth_trial])
    fft_vals = np.fft.fft(condition_1_filt[channel][nth_trial])[:len_data//2]
    # take the last half of the data where there are only possible values (this
    # didnt work with the raw data)
    fft_freqs = np.fft.fftfreq(len_data, 1/fs)[:len_data//2]

    axs[i, 1].plot(fft_freqs, np.abs(fft_vals))
    len_data = len(condition_2_filt[channel][nth_trial])
    fft_vals = np.fft.fft(condition_2_filt[channel][nth_trial])[:len_data//2]
    fft_freqs = np.fft.fftfreq(len_data, 1/fs)[:len_data//2]

    axs[i, 1].plot(fft_freqs, np.abs(fft_vals))
    axs[i, 1].set_xlim(0, 60)
    axs[i, 1].set_title(channel)

for ax in axs.flat:
    ax.set(xlabel='Frequencies (Hz)', ylabel='Amplitude')

plt.subplots_adjust(hspace=0.5)
plt.show()

### Welch's method

Welch's method is an approach to estimate the power spectral density (PSD) of a signal. The PSD is the distribution of power into frequency components. Welch's method reduces noise in the estimated power spectra over the standard periodogram spectrum, but does reduce the frequency resolution. The obtained PSD can be used as a feature in the classification of the EEG data.

In [ ]:
fig, axs = plt.subplots(len(channel_names), 2, figsize=(15, 3 *len(channel_names)))

for i, channel in enumerate(channel_names):
    freqs, psd = signal.welch(condition_1_raw[channel][nth_trial], fs=fs)
    axs[i, 0].plot(freqs, psd, label="eyes closed")
    freqs, psd = signal.welch(condition_2_raw[channel][nth_trial], fs=fs)
    axs[i, 0].plot(freqs, psd, label="eyes open")
    axs[i, 0].set_title("raw " + channel)
    axs[i, 0].set_xlim(0, 100)

    freqs, psd = signal.welch(condition_1_filt[channel][nth_trial], fs=fs)
    axs[i, 1].plot(freqs, psd, label="eyes closed")
    freqs, psd = signal.welch(condition_2_filt[channel][nth_trial], fs=fs,)
    axs[i, 1].plot(freqs, psd, label="eyes open")
    axs[i, 1].set_title("filt " + channel)
    axs[i, 1].set_xlim(0, 70)


for ax in axs.flat:
    ax.set(xlabel='Frequencies (Hz)', ylabel='Amplitude')


plt.subplots_adjust(hspace=0.5)
plt.show()

<h2>Feature extraction</h2> <a class="feature" id="feature"></a>

There are multiple features that can be extracted from EEG data. We will use CSP. 

The goal of CSP is to find a set of spatial filters that can effectively differentiate between two classes of signals based on their covariance matrices. The CSP algorithm is a supervised algorithm, which means that it needs labeled data to train the algorithm. The algorithm will then find a set of spatial filters that can differentiate between the two classes. The algorithm will find a set of spatial filters that will maximize the variance of one class and minimize the variance of the other class. This process results in new features (components) that are linear combinations of the original channels. The goal of the spatial filter is to find a set of spatial weights that maximally discriminate between two or more classes of EEG data.


Because this dataset contained only ten trials for each person we will do the CSP feature extraction on all the subjects. But this way can be easily translated to one person.

In [ ]:
condition_1_raw = defaultdict(lambda: [])
condition_2_raw = defaultdict(lambda: [])
condition_1_filt = defaultdict(lambda: [])
condition_2_filt = defaultdict(lambda: [])


for subject in subject_files:
    idx_condition_1, idx_condition_2, time_stamps, eeg_measurements = extract_columns(subject, data_directory)

    for nth_channel in range(len(channel_names)):

        channel_data = eeg_measurements[:, nth_channel]
        for idx_1, idx_2 in zip(idx_condition_1, idx_condition_2):

            condition_1_raw[channel_names[nth_channel]].append(channel_data[idx_1:idx_1 + fs * 10])
            condition_1_filt[channel_names[nth_channel]].append(signal.lfilter(b, a, signal.sosfilt(sos, channel_data)[idx_1:idx_1 + fs * 10]))

            condition_2_raw[channel_names[nth_channel]].append(channel_data[idx_2:idx_2 + fs * 10])
            condition_2_filt[channel_names[nth_channel]].append(signal.lfilter(b, a, signal.sosfilt(sos, channel_data)[idx_2:idx_2 + fs * 10]))


For the CSP feature extraction we will use a function from the mne library which contains multiple functions for the processing and feature extraction of EEG data. Even though the documentation specifies that the fit_transform function expects the data to be 2D, this function calls another function which is called fit which needs the data to be 3D.

In [ ]:
# the shape of the data is nchannels, ntrials, nsamples so the data needs to be transposed ntrials, nchannels, nsamples
x_condition_1_filt = np.array([condition_1_filt[channel] for channel in channel_names]).transpose(1, 0, 2)
# needs to have the same length
y_condition_1_filt = np.zeros(x_condition_1_filt.shape[0])

x_condition_2_filt = np.array([condition_2_filt[channel] for channel in channel_names]).transpose(1, 0, 2)
y_condition_2_filt = np.ones(x_condition_2_filt.shape[0])

x_condition_1_raw = np.array([condition_1_raw[channel] for channel in channel_names]).transpose(1, 0, 2)
y_condition_1_raw = np.zeros(x_condition_1_raw.shape[0])

x_condition_2_raw = np.array([condition_2_raw[channel] for channel in channel_names]).transpose(1, 0, 2)
y_condition_2_raw = np.ones(x_condition_2_raw.shape[0])

X_train_filt = np.concatenate([x_condition_1_filt, x_condition_2_filt])
y_train_filt = np.concatenate([y_condition_1_filt, y_condition_2_filt])

X_train_raw = np.concatenate([x_condition_1_raw, x_condition_2_raw])
y_train_raw = np.concatenate([y_condition_1_raw, y_condition_2_raw])

n_components = 2
csp_model = CSP(n_components=n_components, reg=None, log=None, norm_trace=False)

# # returns n_samples, n_features_new
x_train_csp_filt = csp_model.fit_transform(X_train_filt, y_train_filt)
x_train_csp_raw = csp_model.fit_transform(X_train_raw, y_train_raw)

# class 0
plt.scatter(x_train_csp_filt[:, 0][y_train_filt == 0], x_train_csp_filt[:, 1][y_train_filt == 0], label='eyes closed')
# class 1
plt.scatter(x_train_csp_filt[:, 0][y_train_filt == 1], x_train_csp_filt[:, 1][y_train_filt == 1], alpha=0.7, label='eyes open')
plt.title('The different classes after CSP feature extraction with the filtered data')
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.legend()
plt.show()

# class 0
plt.scatter(x_train_csp_raw[:, 0][y_train_raw == 0], x_train_csp_raw[:, 1][y_train_raw == 0], label='eyes closed')
# class 1
plt.scatter(x_train_csp_raw[:, 0][y_train_raw == 1], x_train_csp_raw[:, 1][y_train_raw == 1], alpha=0.3, label='eyes open')
plt.title('The different classes after CSP feature extraction with the raw data')
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.legend()
plt.show()

In [ ]:
# setting a random state so the experiment can be replicated
X_train_filt_s, X_test_filt_s, y_train_filt_s, y_test_filt_s = train_test_split(X_train_filt, y_train_filt, test_size=0.2, shuffle=True)
X_train_raw_s, X_test_raw_s, y_train_raw_s, y_test_raw_s = train_test_split(X_train_raw, y_train_raw, test_size=0.2, shuffle=True)

X_train_csp_filt = csp_model.fit_transform(X_train_filt_s, y_train_filt_s)
# class 0
plt.scatter(X_train_csp_filt[:, 0][y_train_filt_s == 0], X_train_csp_filt[:, 1][y_train_filt_s == 0], label='eyes closed')
# class 1
plt.scatter(X_train_csp_filt[:, 0][y_train_filt_s == 1], X_train_csp_filt[:, 1][y_train_filt_s == 1], alpha=0.7, label='eyes open')
plt.title('Training set with the filtered data')
plt.legend()
plt.show()


X_train_csp_raw = csp_model.fit_transform(X_train_raw_s, y_train_raw_s)
# class 0
plt.scatter(X_train_csp_raw[:, 0][y_train_raw_s == 0], X_train_csp_raw[:, 1][y_train_raw_s == 0], label='eyes closed')
# class 1
plt.scatter(X_train_csp_raw[:, 0][y_train_raw_s == 1], X_train_csp_raw[:, 1][y_train_raw_s == 1], alpha=0.3, label='eyes open')
plt.title('Training set with the raw data')
plt.legend()
plt.show()


X_test_csp_filt = csp_model.fit_transform(X_test_filt_s, y_test_filt_s)
# class 0
plt.scatter(X_test_csp_filt[:, 0][y_test_filt_s == 0], X_test_csp_filt[:, 1][y_test_filt_s == 0], label='eyes closed')
# class 1
plt.scatter(X_test_csp_filt[:, 0][y_test_filt_s == 1], X_test_csp_filt[:, 1][y_test_filt_s == 1], alpha=0.7, label='eyes open')
plt.title('Test set with the filtered data')
plt.legend()
plt.show()
X_test_csp_raw = csp_model.fit_transform(X_test_raw_s, y_test_raw_s)
# class 0
plt.scatter(X_test_csp_raw[:, 0][y_test_raw_s == 0], X_test_csp_raw[:, 1][y_test_raw_s == 0], label='eyes closed')
# class 1
plt.scatter(X_test_csp_raw[:, 0][y_test_raw_s == 1], X_test_csp_raw[:, 1][y_test_raw_s == 1], alpha=0.7, label='eyes open')
plt.title('Test set with the raw data')
plt.legend()
plt.show()

Here we are calculating the accuracy of the filtered model and the unfiltered model. The accuracy is calculated by dividing the number of correct predictions by the total number of predictions. The accuracy of the filtered model is higher than the accuracy of the unfiltered model. This is because the filtered model has less noise in the data. So the model can make better predictions.

To calculate the accuracy, we will use three different classification methods. These methods are Knn (K nearest neighbors), SVM (Support Vector Machines) and LDA (Linear Discriminant Analysis). These methods are used to classify the EEG data. The Knn method is a method that classifies the data based on the nearest neighbors. The SVM method is a method that classifies the data based on a hyperplane. The LDA method is a method that classifies the data based on the linear combination of features.

In [ ]:
svm = SVC(kernel='linear')
lda = LinearDiscriminantAnalysis()
knn = KNeighborsClassifier(n_neighbors=3)

svm.fit(X_train_csp_filt, y_train_filt_s)
svm_test_predictions = svm.predict(X_test_csp_filt)
print("Accuracy SVM filtered: ", accuracy_score(y_test_filt_s, svm_test_predictions))

svm.fit(X_train_csp_raw, y_train_raw_s)
svm_test_predictions = svm.predict(X_test_csp_raw)
print("Accuracy SVM raw: ", accuracy_score(y_test_raw_s, svm_test_predictions))

lda.fit(X_train_csp_filt, y_train_filt_s)
lda_test_predictions = lda.predict(X_test_csp_filt)
print("Accuracy LDA filtered: ", accuracy_score(y_test_filt_s, lda_test_predictions))

lda.fit(X_train_csp_raw, y_train_raw_s)
lda_test_predictions = lda.predict(X_test_csp_raw)
print("Accuracy LDA raw: ", accuracy_score(y_test_raw_s, lda_test_predictions))

knn.fit(X_train_csp_filt, y_train_filt_s)
knn_test_predictions = knn.predict(X_test_csp_filt)
print("Accuracy KNN filtered: ", accuracy_score(y_test_filt_s, knn_test_predictions))

knn.fit(X_train_csp_raw, y_train_raw_s)
knn_test_predictions = knn.predict(X_test_csp_raw)
print("Accuracy KNN raw: ", accuracy_score(y_test_raw_s, knn_test_predictions))